<a href="https://colab.research.google.com/github/Jeongho-K/Car-model-classification/blob/main/data/2_Car_classification_with_efficient_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car classification

data: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=554

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 차량 방향 확인할 이미지 리스트 생성

In [ ]:
import shutil
import os
import cv2
import matplotlib.pyplot as plt
import glob

In [ ]:
def make_img_list(BASE_FOLDER, OUTPUT):
    f_list = []
    for model_name in os.listdir(BASE_FOLDER):
        model_path = os.path.join(BASE_FOLDER, model_name)
        for detail_forder in os.listdir(model_path):
            folder_path = os.path.join(model_path, detail_forder + '/*')
            f_list += glob.glob(folder_path)

    with open(OUTPUT, 'w', encoding="UTF8") as f:
        for line in f_list:
            f.write(line)
            f.write('\n')

In [ ]:
train_BASE_FOLDER = r"/content/drive/MyDrive/Car-model-classification/data/1.Training/원천데이터/HY_현대"
train_OUTPUT = '/content/drive/MyDrive/Car-model-classification/data/train_image_list.txt'

make_img_list(train_BASE_FOLDER, train_OUTPUT)

In [ ]:
val_BASE_FOLDER = r"/content/drive/MyDrive/Car-model-classification/data/2.Validation/원천데이터/HY_현대"
val_OUTPUT = '/content/drive/MyDrive/Car-model-classification/data/val_image_list.txt'

make_img_list(val_BASE_FOLDER, val_OUTPUT)

## yolov4 custom(차량 앞, 뒤, 옆 분류)

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.17 MiB | 15.10 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.


- darknet모델내 makefile파일 내용 수정  
- GPU=1, CUDNN=1, OPENCV=1

In [ ]:
import os
path = '/content/darknet'
os.chdir(path)
!make

In [ ]:
# verify installation
!./darknet

usage: ./darknet <function>


In [ ]:
# pretrained weights
import shutil
shutil.copy('/content/drive/MyDrive/Car-model-classification/models/YOLOv4_사전학습/yolov4-car.cfg', 'cfg/yolov4-car.cfg')
shutil.copy('/content/drive/MyDrive/Car-model-classification/models/YOLOv4_사전학습/yolov4-car_best.weights', 'cfg/yolov4-car_best.weights')
shutil.copy('/content/drive/MyDrive/Car-model-classification/models/YOLOv4_사전학습/car.names', 'data/car.names')
shutil.copy('/content/drive/MyDrive/Car-model-classification/models/YOLOv4_사전학습/car.data', 'data/car.data')

'data/car.data'

In [ ]:
!./darknet detector test data/car.data cfg/yolov4-car.cfg cfg/yolov4-car_best.weights -dont_show -out /content/drive/MyDrive/Car-model-classification/data/train_direction_results.json < /content/drive/MyDrive/Car-model-classification/data/train_image_list.txt

In [ ]:
!./darknet detector test data/car.data cfg/yolov4-car.cfg cfg/yolov4-car_best.weights -dont_show -out /content/drive/MyDrive/Car-model-classification/data/val_direction_results.json < /content/drive/MyDrive/Car-model-classification/data/val_image_list.txt

### 차량 front, side, back label
yolo모델로 생성된 direction_results.json으로 방향 class별 3개 라벨 데이터 생성

## 이미지 crop

In [ ]:
import os
import json
import cv2
import numpy as np
import shutil

In [ ]:
# 텍스트 읽기
def read_text(file_path, encoding="utf-8"):
    text = None
    with open(file_path, 'r', encoding=encoding) as f:
        text = f.read()
    return text

In [ ]:
def load_json(json_path, encoding="utf-8"):
    return json.loads(read_text(json_path, encoding))

In [ ]:
def check_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
# box 정보에 대해 margin 값만큼 확장함
def expand_box(box, margin, img_width = None, img_height = None):
    x = box["left"]
    y = box["top"]
    w = box["width"]
    h = box["height"]

    x = x - margin
    x = max(x, 0)
    y = y - margin
    y = max(y, 0)

    w = w + 2*margin
    if img_width and x + w > img_width:
        w = img_width - x
    
    h = h + 2*margin
    if img_height and y + h > img_height:
        h = img_height - y

    new_box = {
        "left": x, 
        "top": y,
        "width": w,
        "height": h
    }

    return new_box

In [ ]:
# 이미지 crop
def crop_image(image_np, box, margin = 0):
    if margin > 0:
        box = expand_box(box, margin)
       
    x1 = box["left"]
    y1 = box["top"]
    x2 = box["left"] + box["width"]
    y2 = box["top"] + box["height"]
    return image_np[y1:y2, x1:x2]

In [ ]:
# 한글 경로 지원
def imread( file_path ) :
    stream = open( file_path.encode("utf-8") , "rb")
    bytes = bytearray(stream.read())
    np_array = np.asarray(bytes, dtype=np.uint8)
    return cv2.imdecode(np_array , cv2.IMREAD_UNCHANGED)

In [ ]:
# 한글 경로 지원
def imwrite(file_path, img, params=None):
    try: 
        ext = os.path.splitext(file_path)[1]
        result, n = cv2.imencode(ext, img, params)
        if result: 
            with open(file_path, mode='w+b') as f:
                n.tofile(f)
                return True
        else: 
            return False 
    except Exception as e: 
        print(e)
        return False

In [ ]:
# 전체 이미지 bbox가 있는 경우 crop해서 방향정보에 따라 저장 
def crop_store_by_direction(direction_path, BASE_OUTPUT_FOLDER):
    dir_json = load_json(direction_path)

    for dir in dir_json:

        if len(dir['objects']) > 0:
            confidence = [x['confidence'] for x in dir['objects']]
            best_idx = confidence.index(max(confidence)) # 가장 신뢰도 높은 분류 class

            img_path = dir['filename']
            tmp_list = img_path.split('/')
            tmp_list[7] = '라벨링데이터'
            img_name = tmp_list[11]
            json_name = img_name.split('.')[0] + '.json'
            json_path = '/'.join(tmp_list[:-1]) + '/' + json_name

            img_json = load_json(json_path)
            object_list = img_json["learningDataInfo"]["objects"]
            
            

            for i in range(len(object_list)):
                if img_json["learningDataInfo"]["objects"][i]["classId"] == 'P00.차량전체':
                    # 차량 전체 bbox
                    box = {
                            "left": int(img_json["learningDataInfo"]["objects"][i]["left"]),
                            "top": int(img_json["learningDataInfo"]["objects"][i]["top"]),
                            "width": int(img_json["learningDataInfo"]["objects"][i]["width"]),
                            "height": int(img_json["learningDataInfo"]["objects"][i]["height"])
                            }

                    # 이미지 크롭
                    img = imread(img_path)
                    cropped_img = crop_image(img, box, 10)

                    # 방향 확인 후 해당 폴더에 저장
                    direction = dir['objects'][best_idx]['name']
                    output_folder = os.path.join(BASE_OUTPUT_FOLDER, direction)
                    check_folder(output_folder)
                    output_path = os.path.join(output_folder, str(img_name))

                    imwrite(output_path, cropped_img)

                    # 라벨링 데이터도 같이 이동
                    shutil.copyfile(json_path, os.path.join(output_folder, str(json_name)))

        print(dir['frame_id'])

In [ ]:
# train & validation img
train_val_direction_path = '/content/drive/MyDrive/Car-model-classification/data/train_direction_results.json'
train_val_output_folder = '/content/drive/MyDrive/Car-model-classification/data/cropimg/train_val'

crop_store_by_direction(train_val_direction_path, train_val_output_folder)

In [ ]:
# test img
val_direction_path = '/content/drive/MyDrive/Car-model-classification/data/val_direction_results.json'
val_output_folder = '/content/drive/MyDrive/Car-model-classification/data/cropimg/test'

crop_store_by_direction(val_direction_path, val_output_folder)